In [ ]:
@contents @fj
1 e --- error handling
1 s --- state
1 w --- workers
123 p --- pages
1 a --- application
1 r --- render

http://localhost:8888/static/p/gchatstats/upload.html

In [44]:
import nbformat
import extensions.my_ext

# import importlib
# importlib.reload(extensions.my_ext)

extensions.my_ext.load_ipython_extension(get_ipython())

def make(fname):
    source = open(fname, 'rb').read()
    nb = nbformat.reads(source.decode('utf8'), as_version=4)
    parents = {}
    cells = [extensions.my_ext.NotebookCell(cell, parents, index) for index, cell in enumerate(nb.cells)]
    with open('/data/sets/server/gchatstats/static/upload.html', 'w') as f:
        f.write(make_html(cells))

def make_html(cells):
    header_html = ''
    body_html = ''
    jsx_scripts = ''
    scripts = ''
    run_scripts = ''
    global_error = ''
    for cell in cells:
        if cell.magic == '%%js' and '@out' in cell.labels:
            scripts += cell.get_output('application/javascript') + '\n'
        if cell.magic == '%%js' and '@out_run' in cell.labels:
            run_scripts += cell.get_output('application/javascript') + '\n'
        if cell.magic == '%%js' and '@out_error' in cell.labels:
            global_error += cell.source #('application/javascript') + '\n'
        elif cell.magic == '%%jsx' and '@out' in cell.labels:
            jsx_scripts += cell.get_output('application/javascript') + '\n'
        elif cell.magic == '%%css' and '@out' in cell.labels:
            header_html += cell.get_output('text/html') + '\n'
        elif cell.magic == '%%html' and '@header' in cell.labels:
            header_html += cell.source + '\n'
        elif cell.magic == '%%html' and '@body' in cell.labels:
            body_html += cell.source + '\n'
        elif cell.kind == 'code' and cell.source.startswith('%css '):
            header_html += cell.get_output('text/html') + '\n'
        elif cell.magic == '%%requirejs':
            require_config = cell.get_output('application/javascript') + '\n'
    res = '<!DOCTYPE html><html><head>\n'
    res += header_html + '\n'
    if hasattr(extensions.my_ext, 'requirejs'):
        res += '<script src=\"%s\"></script>\n' % extensions.my_ext.requirejs['lib_path']
    res += '<script>\n' + require_config + '\n</script>\n'
    res += '<script>\n' + global_error + '\n</script>\n'
    res += '<script>\n' + scripts + '\n</script>\n'
    res += '<script>\n' + jsx_scripts + '\n</script>\n'
    res += '</head><body>\n'
    res += body_html
    res += '<script>\n' + run_scripts + '\n</script>\n'
    res += '</body></html>\n'
    return res

In [45]:
%%requirejs /static/requirejs@2.3.1/require.min.js
$           jquery  /static/jquery@2.1.1/jquery.min.js
React       react   /static/react@15.3.1/build/react.min.js
ReactDOM            /static/react@15.3.1/build/react-dom.min.js
R                   /static/ramda@0.22.1/ramda.min.js

<IPython.core.display.Javascript object>

In [46]:
%%js @out_error
require.config({
  waitSeconds : 60
})

<IPython.core.display.Javascript object>

In [47]:
%%js @out_run
if (window.location.hostname == 'localhost') {
    if (window.IPython) {
        window.server_path = '';//http://localhost:8080';        
    } else {
        window.server_path = '';//http://localhost:8088';
    }
} else {
    window.server_path = '';//http://localhost:8080';
}
if (!window.IPython) {
    $(function() {
      reset();
      render();
    });
}

<IPython.core.display.Javascript object>

In [48]:
%%js @out_error @1e

// http://stackoverflow.com/questions/5916900/how-can-you-detect-the-version-of-a-browser
window.get_browser = function(){
    var ua=navigator.userAgent,tem,M=ua.match(/(opera|chrome|safari|firefox|msie|trident(?=\/))\/?\s*(\d+)/i) || []; 
    if(/trident/i.test(M[1])){
        tem=/\brv[ :]+(\d+)/g.exec(ua) || []; 
        return {name:'IE',version:(tem[1]||'')};
        }   
    if(M[1]==='Chrome'){
        tem=ua.match(/\bOPR\/(\d+)/)
        if(tem!=null) {return {name:'Opera', version:tem[1]};}
        }   
    M=M[2]? [M[1], M[2]]: [navigator.appName, navigator.appVersion, '-?'];
    if((tem=ua.match(/version\/(\d+)/i))!=null) {M.splice(1,1,tem[1]);}
    return M[0] + ' ' + M[1];
 }

function get_debug_info() {
    return {
        hash: window.location.hash,
        ipython: window.IPython,
        browser: get_browser(),
        state: window.state
    }
}

window.onerror = function(msg, url, line, col, error) {
    var message = "Error: " + msg + "\nurl: " + url + "\nline: " + line ;
    message += !col ? '' : '\ncolumn: ' + col;
    message += !error ? '' : '\nerror: ' + error;
    console.log(message);
    $.post(server_path + '/error', {
        page : window.location.href.split('#')[0],
        message : message,
        info : JSON.stringify(get_debug_info()),
    }, function() {
        alert('Произошла ошибка.\nИнформация отправлена администратору.\nПопробуйте перезагрузить страницу и повторить запрос.')
    }).fail(function() {
        alert('Не удается связаться с сервером.\n\
Проверьте соединение и перезагрузите эту страницу.\n\
Если соединение не прерывалось, то это проблема на стороне сервера.\n\
В этом случае напишите на pallada-92@ya.ru');
    });
    return true;
};

<IPython.core.display.Javascript object>

In [49]:
%%html @header
<meta charset="utf-8">
<title>Проанализировать групповой чат VK!</title>
<link href="https://fonts.googleapis.com/css?family=Roboto:400,300,500,700&subset=latin,cyrillic" rel="stylesheet" type="text/css">

In [50]:
%%css .css1 @out
.root {
    font-family: Roboto, Helvetica;
    color: #333333;
    background-color: white;
}
.container {
    position: relative;
    width: 800px;
    left: 50%;
    margin-left: -400px;
}
.link {
    color: #5555EE;
    cursor: pointer;
    -webkit-transition: color 0.3s;
    -moz-transition: color 0.3s;
    -o-transition: color 0.3s;
    transition: color 0.3s
}
.link:hover {
    color: #CC0000;
}
.error {
    color: #CC0000;
    margin: 10px;
}
li {
    margin-bottom: 30px;
}
h1 {
    text-align: center;
    font-size: 20px;
    margin-bottom: 40px;
    margin-top: 30px;
    color: #333333;
}

In [51]:
%%css .css1 @out
.remarks {
    color: gray;
    width: 100%;
    padding-top: 40px;
    text-align: left;
}
.remark {
    padding: 10px 0px;
    display: inline-block;
    width: 100%;
    text-align: left;
}
.remark2 {
    padding: 10px 0px;
    display: inline-block;
    width: 50%;
    text-align: left;
}

In [52]:
%%jsx @out @1s
window.reset = function() {
    window.consts = {
        chat_names_step: 200,
        messages_step: 200,
        messages_limit: 2000
    };
    window.secret = {
        token: undefined,    
    };
    window.state = {
        page: 1,
        user_id: undefined,
        chat_id: undefined,
        chat_names_offset: 0,
        chat_names_get: false,
        chat_names_count: undefined,
        hash: Math.round(Math.random() * 10000000000),
        messages_get_offset: 0,
        messages_post_offset: 0,
        messages_all_count: undefined,
        messages_count: undefined,
        page3: 'get_messages' // 'post_messages' / 'get_users' / 'post_users' / 'finished'
    };
    window.cache = {
        chat_names: [],
        vk_requests_queue: [],
    }
}

window.generic_vk_request = function (method, params, ok_handler) {
    let url = 'https://api.vk.com/method/' + method + '?callback=?';
    $.getJSON(url, params).done(function(data, text){
        if ('response' in data) {
            ok_handler(data['response']);
        } else {
            throw JSON.stringify({url, data});
        };
    }).fail(function(jq, text, exception) {
        let info = {jq, text, exception, url};
        throw JSON.stringify(info);
    });
}

window.generic_server_request = function (method, params, ok_handler) {
    let url = server_path + '/' + method;
    $.post(url, params).done(function(data, text){
        ok_handler(data);
    }).fail(function(jq, text, exception) {
        let info = {jq, text, exception, url};
        throw JSON.stringify(info);
    });
}

window.get_chat_names = function() {
    generic_vk_request('messages.getDialogs', {
        access_token: secret.token,
        offset: state.chat_names_offset,
        count: consts.chat_names_step,
        v: '5.38',
    }, (response) => {
        state.chat_names_count = response.count;
        state.chat_names_offset += consts.chat_names_step;
        console.log(response);
        // Group chats are those, which have users_count field
        let group_chats = R.filter(
            R.pathSatisfies(
              R.complement(R.equals(undefined)),
              ['message', 'users_count']
            ),
            response.items
        );
        // For each group chat, get pair [chat_id, title] and save it to state.chat_names
        cache.chat_names = cache.chat_names.concat(R.map(
            R.juxt([R.path(['message', 'chat_id']), R.path(['message', 'title'])]),
            group_chats
        ));
        if (state.chat_names_offset < state.chat_names_count && state.page == 2) {
            setTimeout(get_chat_names, 500);
        } else if (state.chat_names_offset >= state.chat_names_count) {
            state.chat_names_get = true;
        }
        render();
    });
}

window.get_users = function() {
    state.page3 = 'get_users';
    generic_vk_request('messages.getChatUsers', {
        access_token: secret.token,
        chat_id: state.chat_id,
        fields: 'screen_name, photo_50, photo_100, photo_200_orig',
        v: '5.38',
    }, (response) => {
        console.log(response);
        cache.chat_users = response;
        if (state.page == 3) {
            setTimeout(post_users, 300);
        }
        render();
    });
}

window.post_users = function() {
    state.page3 = 'post_users';
    console.log(JSON.stringify(cache.chat_users));
    generic_server_request('upload_vk_users', {
        user_id: state.user_id,
        chat_id: state.chat_id,
        chat_name: R.find(R.propEq(0, state.chat_id), cache.chat_names)[1],
        hash: state.hash,
        users_data: JSON.stringify(cache.chat_users),
    }, (response) => {
        get_messages();
        render();
    });
}

window.get_messages = function() {
    state.page3 = 'get_messages';
    console.log('get_messages', state.chat_id);
    generic_vk_request('messages.getHistory', {
        access_token: secret.token,
        offset: state.messages_get_offset,
        count: consts.messages_step,
        peer_id: 2000000000 + state.chat_id,
        v: '5.38',
    }, (response) => {
        console.log('resp', response);
        state.messages_all_count = response.count;
        state.messages_count = Math.min(state.messages_all_count, consts.messages_limit);
        console.log('resp1');
        state.messages_get_offset += consts.messages_step;
        cache.messages = response.items;
        console.log('setting timeout1', state.page);
        if (state.page == 3) {
            console.log('setting timeout');
            setTimeout(post_messages, 300);
        }
        render();
    });
}

window.post_messages = function() {
    state.page3 = 'post_messages';
    generic_server_request('upload_vk_messages', {
        offset: state.messages_offset,
        hash: state.hash,
        messages: JSON.stringify(cache.messages),
    }, (response) => {
        state.messages_post_offset += consts.messages_step;
        let limit_reached = state.messages_get_offset >= state.messages_count;
        if (!limit_reached && state.page == 3) {
            setTimeout(get_messages, 300);
        } else if (limit_reached && state.page == 3) {
            state.messages_loaded = true;
            setTimeout(finalize_upload, 300);
        }
        render();
    })
}

window.finalize_upload = function() {
    state.page3 = 'finalizing';
    render();
    generic_server_request('upload_vk_finalize', {
        hash: state.hash,
    }, (response) => {
        state.page3 = 'finalized';
        render();
        window.location = server_path + '/vk' + state.hash
    });
}

window.check_addr_syntax = function (addr) {
  var s1 = "https://oauth.vk.com/blank.html#access_token=";
  var s2 = "&expires_in=";
  var s3 = "&user_id=";
  var p1 = addr.indexOf(s1);
  if (p1 == -1) return {error: "Адрес должен начинаться с \"https://oauth.vk.com/blank.html#access_token=\""};
  var p2 = p1 + s1.length;
  var p3 = addr.indexOf(s2);
  if (p3 == -1) return {error: "В адресе должно быть поле \"&expires_in=\""};
  var token = addr.slice(p2, p3);
  var p4 = p3 + s2.length;
  var p5 = addr.indexOf(s3);
  if (p5 == -1) return {error: "В адресе должно быть поле \"&user_id=\""};
  var expires = addr.slice(p4, p5);
  var p6 = p5 + s3.length;
  var p7 = addr.length;
  var user_id = addr.slice(p6, p7);
  if (!(/^[0-9a-f]*$/.test(token)) || token == '') return {error: "Некорректный токен \"" + token+ "\""};
  if (!(/^[0-9]*$/.test(user_id)) || user_id == '') return {error: "Некорректный user_id \"" + user_id + "\""};
  return {token, user_id};
}

Starting react...


<IPython.core.display.Javascript object>

In [53]:
%%jsx @out
window.Action = React.createClass({
  render: function() {
    return <div style={{textAlign: 'center', margin: 20}}>
      <span className="link" onClick={this.props.handler}>
        {this.props.text} 
      </span>
    </div>;
  }
});

window.Actions = React.createClass({
  render: function() {
    let actions = R.map(
      ([id, title]) => <Action handler={this.props.handler.bind(this, id)} text={title} key={id}/>,
      this.props.list
    );
    return <div>{actions}</div>;
  }
});

window.Errors = React.createClass({
  render: function() {
    let errors = R.map(
      (error) => <div key={error} className="error">{error}</div>,
      this.props.data
    );
    return (
      <div>
        {errors}
      </div>
    );
  }
});

window.Progress = React.createClass({
  render: function() {
    let preload_percent = this.props.preload * 100,
        load_percent = this.props.value * 100;
    let outer_style = {
      height: 25,
      borderRadius: 5,
      border: '1px solid gray',
      position: 'relative',
      overflow: 'hidden'
    }, preload_style = {
      position: 'absolute',
      backgroundColor: 'rgb(228, 243, 243)',
      width: preload_percent + '%',
      height: 25,
      top: 0,
      left: 0,
    }, load_style = {
      position: 'absolute',
      backgroundColor: 'lightblue',
      width: load_percent + '%',
      height: 25,
      top: 0,
      left: 0,
    }, text_style = {
      fontSize: 15,
      position: 'absolute',
      color: '#555555',
      top: 3,
      left: 0,
      width: '100%',
      height: 20,
      textAlign: 'center'
    }
    return (
      <div style={outer_style}>
        <div style={preload_style}/>
        <div style={load_style}/>
        <div style={text_style}>{this.props.text}</div>
      </div>
    );
  }
});

window.Textarea = React.createClass({
  render: function() {
    let handler = (event) => {
        this.props.handler(event.target.value);
    };
    let style = {minWidth: '95%', minHeight: 50, padding: 5};
    return (
        <textarea style={style} ref="textarea" onChange={handler} value={this.props.value}/>
    );
  }
});

window.FirstPage = React.createClass({
    getInitialState: function() {
        return {value: '', errors: []};
    },
    valChange: function(value) {
        let errors = [];
        let parse_res = check_addr_syntax(value);
        if ('error' in parse_res) {
            if (value != '') {
                errors.push(parse_res.error);
                this.setState({value, errors});
            }
        } else {
            state.page = 2;
            secret.token = parse_res.token;
            state.user_id = parse_res.user_id;
            get_chat_names();
            render();
        }
    },
    render: function() {
        let href = "https://oauth.vk.com/authorize?client_id=4910996&redirect_uri=";
        href += "https://oauth.vk.com/blank.html&display=page&scope=4096&response_type=token";
        return (
          <div>
          <h1>Получение токена для анализа групповых чатов</h1>
          <ul>
          <li>Перейдите по&nbsp;
            <a href={href}
             target="_blank" className="link">ссылке</a> в новом окне.</li>
          <li>Разрешите приложению GroupChatStats доступ к работе с личными сообщениями.</li>
          <li> Вы должны увидеть текст "Пожалуйста, не копируйте данные из адресной строки..."</li>
          <li> Скопируйте данные из адресной строки, закройте окно и вставьте их сюда:<br/><br/>
            <Textarea handler={this.valChange} value={this.state.value}/>
            <Errors data={this.state.errors}/> 
          </li>
          </ul>
          <div className="remarks">
          <div className="remark">
          Токен не передается на сервер и используется исключительно для чтения истории выбранного группового чата.
          </div>
          </div>
          </div>
        );
    }
});

window.SecondPage = React.createClass({
  render: function() {
    let handler = (chat_id) => {
        state.page = 3;
        state.chat_id = chat_id;
        get_users();
        render();
    }
    let title = state.chat_names_get ?
        'Выберите чат для анализа' :
        'Идет получение списка чатов...';
    return (
        <div>
        <h1>{title}</h1>
        <Actions handler={handler} list={cache.chat_names}/>
        <div className="remarks">
        <div className="remark">
        </div>
        </div>
        </div>
    );
  }
});

window.ThirdPage = React.createClass({
    render: function() {
        let title = 'Загрузка сообщений на сервер...';
        if (state.page3 == 'finalized') {
            title = 'Сообщения загружены!'
        }
        let preload = state.messages_get_offset / state.messages_count;
        let progress = state.messages_post_offset / state.messages_count;
        let count_text ='(' + state.messages_post_offset + ' / ' + state.messages_count  + ')';
        if (!state.messages_count) {
            count_text = '';
        }
        let prog_text = {
            get_users: 'Получение информации об участниках чата...',
            post_users: 'Отправка информации об участниках чата...',
            get_messages: 'Получение сообщений чата... ' + count_text,
            post_messages: 'Отправка. сообщений чата... ' + count_text,
            finalizing: 'Обработка данных...',
            finalized: 'Завершение',
        }[state.page3];
        return (
            <div>
            <h1>{title}</h1>
            <Progress value={progress} preload={preload} text={prog_text}/>
            <div className="remarks">
            <div className="remark">
            </div>
            </div>                
            </div>
        )
    }
});

window.Application = React.createClass({
  render: function() {
    let page_block = null;
    if (state.page == 1) {
      page_block = <FirstPage/>;
    } else if (state.page == 2) {
      page_block = <SecondPage/>;
    } else if (state.page == 3) {
      page_block = <ThirdPage/>;
    }
    return (
      <div className="container">
        {page_block}
      </div>
    )
  }
});

window.render = function() {
  ReactDOM.render(
    <Application/>,
    document.getElementById('root')
  );
}

<IPython.core.display.Javascript object>

In [ ]:
%%js no_out
reset();
render();

In [54]:
%%html @body
<div class="css1">
    <div id="root" class="root"></div>
</div>